In [ ]:
from functools import *
import random

In [ ]:
# Se le pasa una lista que define la permutación
def pi_inverse(l):
    n = len(l)
    Bn = BraidGroup(n)
    p = PermutationGroupElement(l)
    ciclos = p.cycle_tuples()
    b = Bn([1,-1])
    
    
    # Descomponemos en transposiciones los ciclos
    desc_trans = []
    for i in range(len(ciclos)):
        for j in range(len(ciclos[i])-1,0, -1):
            desc_trans.append((ciclos[i][0], ciclos[i][j]))
    
    desc_trans_sim = []
    for i in range(len(desc_trans)):
        d = [(desc_trans[i][0],desc_trans[i][0]+1)]
        for j in range(desc_trans[i][0]+1,desc_trans[i][1]):
            d = [(j,j+1)] + d + [(j,j+1)]
        desc_trans_sim = desc_trans_sim + d
            
    for i in range(len(desc_trans_sim)-1,-1,-1):          
                b = b * Bn([desc_trans_sim[i][0]])

        
    return b
    
    

In [ ]:
def order_cycles(l):
    mov = True

    while mov:
        mov = False
        for i in range(len(l)-1):
            if len(l[i]) < len(l[i+1]):
                aux = l[i]
                l[i] = l[i+1]
                l[i+1] = aux
                mov = True
    
    return l

In [ ]:
def GuessPermutation(v,w,n):
    vp = PermutationGroupElement(v.permutation()).cycle_tuples(singletons=True)
    wp = PermutationGroupElement(w.permutation()).cycle_tuples(singletons=True)
    
    vp = order_cycles(vp)
    wp = order_cycles(wp)

    
    tau = [i for i in range(n)]
    
    for i in range(len(wp)):
        for j in range(len(wp[i])):
            tau[vp[i][j]-1] = wp[i][j]
    #print(tau)
    return pi_inverse(tau)
    

In [ ]:
Bn = BraidGroup(5)
v = Bn([1])
x = Bn([3])
w = (x^-1)*v*x
GuessPermutation(v,w,5)


In [ ]:
def inf(b_normal,n):
    Bn = BraidGroup(n)
    alpha = Bn([1,-1])
    delta = Bn.delta()
    neutro = Bn([1,-1])

    
    rb = 0
    
    delta_r = b_normal[0]
    #print(delta_r)
    while delta_r != neutro:
        delta_r = delta_r*delta
        rb = rb + 1
        #print("rb", rb)
    #print("SALE")
    
    return rb
    

In [ ]:
def tauC(w,r,n):
    Bn = BraidGroup(n)
    delta = Bn.delta()
    
    return (delta^-r)*w*(delta^r)

In [ ]:
def compact(normal_form, n):
    Bn = BraidGroup(n)
    c = Bn([1,-1])
    
    for i in range(len(normal_form)):
        c = c*normal_form[i]
    
    return c
    


In [ ]:
def breakConjugate(v,w,n):
    Bn = BraidGroup(n)
    delta = Bn.delta()
    alpha = Bn([1,-1])
    v_normal = v.left_normal_form()
    w_normal = w.left_normal_form()
    iteraciones = 0
    max_iteraciones = 100


    
    rw = inf(w_normal,n)
    rv = inf(v_normal,n)
    
    
    while rw < rv and iteraciones < max_iteraciones:
        gamma = delta*tauC(w_normal[1],rw,n)
        alpha = gamma*alpha
        w = gamma*compact(w_normal,n)*(gamma^-1)
        w_normal = w.left_normal_form()
        rw = inf(w_normal,n)
        iteraciones += 1



    sw = len(w_normal)-1 + inf(w_normal,n)
    sv = len(v_normal)-1 + inf(v_normal,n)
    while sw > sv and iteraciones < max_iteraciones:
        gamma = w_normal[len(w_normal)-1]
        alpha = gamma*alpha
        w = gamma*compact(w_normal,n)*(gamma^-1)
        w_normal = w.left_normal_form()
        sw = len(w_normal)-1 + inf(w_normal,n)
        iteraciones += 1
    
    v = compact(v_normal,n)
    w = compact(w_normal,n)
    mu = GuessPermutation(v,w,n)
    alpha = mu*alpha
    w = mu*w*(mu^-1)
    
    if v == w and iteraciones < max_iteraciones:
        return alpha
    else:
        return "failed"
        
        
        
    
    
    
    
    

In [ ]:
Bn = BraidGroup(5)
v = Bn([-1,-3,2,4,-1,-3])
x = Bn([2,4,3,1,3])
w = (x^-1)*v*x
conjugator = breakConjugate(v,w,5)
#print(w == conjugator^-1*v*conjugator)
print("w: ", w.left_normal_form())
print("v: ", v.left_normal_form())
print("x: ", x)
print("conjugador: ", conjugator)

print(GuessPermutation(v,w,5))
print(w == conjugator^-1*v*conjugator)
#print(w.left_normal_form())
#print((conjugator^-1*v*conjugator).left_normal_form())

In [ ]:
def test(lv, lx, n, repeticiones):
    Bn = BraidGroup(n)
    index_generators1 = list(range(-n+1,0)) + list(range(1,n))
    index_generators2 = list(range(1,n))
    
    aciertos = 0
    
    for i in range(repeticiones):
        print("Progreso:", i)
        v_list = random.choices(index_generators1, k = lv)
        x_list = random.choices(index_generators2, k = lx)
        #print("v_list: ", v_list)
        #print("x_list: ", x_list)
        v = Bn(v_list)
        x = Bn(x_list)
       
        w = (x^-1)*v*x
        
        a = breakConjugate(v,w,n)
        
        if a != "failed":
            if (a^-1)*v*a != w:
                print("ERROR")
        
        if breakConjugate(v,w,n) != "failed":
            aciertos = aciertos + 1
    
    print("Aciertos: ", (aciertos*1.0)/repeticiones)
    #print("Fallos: ", repeticiones-aciertos)
    

In [ ]:
def test_general(li,lf, ni, nf, r):
    for i in range(ni,nf+1,5):
        for j in range(li,lf+1,5):
            print("n =", i, "l =", j, ":")
            test(j,j,i,r)

In [ ]:
random.seed(5)

In [ ]:
test_general(5,10,5,25,10)